#  FHIR Integration with Aligned Model Inference

This notebook processes FHIR Observation data and infers CHD risk using a model trained only on features realistically available via FHIR.

In [1]:

import pandas as pd
import requests
import xgboost as xgb
import joblib
from datetime import datetime


In [4]:

# Load model and required feature order
booster = joblib.load("../src/models/tuned_xgb_fhir.pkl")
model_features = joblib.load("/mnt/data/cardiorisk_2505/src/models/fhir_feature_order.pkl")


In [5]:

def fetch_fhir_observations(patient_id="example", server_url="https://hapi.fhir.org/baseR4"):
    url = f"{server_url}/Observation?subject=Patient/{patient_id}"
    r = requests.get(url, headers={"Accept": "application/fhir+json"})
    return r.json() if r.status_code == 200 else {}

bundle = fetch_fhir_observations()


In [14]:

def fhir_observation_to_df(bundle):
    records = []
    for entry in bundle.get("entry", []):
        res = entry.get("resource", {})
        if res.get("resourceType") == "Observation":
            record = {
                "code": res.get("code", {}).get("text", res.get("code", {}).get("coding", [{}])[0].get("code")),
                "value": res.get("valueQuantity", {}).get("value", None),
                "unit": res.get("valueQuantity", {}).get("unit", None),
                "date": res.get("effectiveDateTime", None)
            }
            records.append(record)
    return pd.DataFrame(records)

obs_df = fhir_observation_to_df(bundle)
obs_df.head(50)

,code,value,unit,date
0,Heart rate,44.0,beats/minute,1999-07-02
1,29463-7,185.0,lbs,2016-03-28
2,29463-7,185.0,lbs,2016-03-28
3,29463-7,185.0,lbs,2016-03-28
4,29463-7,185.0,lbs,2016-03-28
5,29463-7,185.0,lbs,2016-03-28
6,29463-7,185.0,lbs,2016-03-28
7,Body temperature,36.5,C,1999-07-02
8,Body temperature,36.5,C,1999-07-02
9,Body temperature,36.5,C,1999-07-02


In [7]:

# Clean and map to feature names
code_map = {
    '29463-7': 'Weight',
    '8480-6': 'sysBP',
    '8462-4': 'diaBP',
    '2093-3': 'totChol',
    '2345-7': 'glucose',
    '8867-4': 'Heart Rate',
    '8310-5': 'Body Temperature'
}
obs_df['code'] = obs_df['code'].map(code_map).fillna(obs_df['code'])
obs_df = obs_df.dropna(subset=['value', 'date'])
obs_df['date'] = pd.to_datetime(obs_df['date'])

latest_obs = obs_df.sort_values('date').drop_duplicates(subset='code', keep='last')
pivoted_df = latest_obs.set_index("code")["value"].to_frame().T


In [8]:

# Add dummy age value (FHIR birthDate not fetched here)
pivoted_df["age"] = 55  # placeholder; replace with patient age if known


In [ ]:

# Align to model feature order
for col in model_features:
    if col not in pivoted_df.columns:
        pivoted_df[col] = 0
pivoted_df = pivoted_df[model_features]

code,age,sysBP,diaBP,totChol,glucose
value,55,0,0,0,0


In [11]:

# Predict
dmatrix_input = xgb.DMatrix(pivoted_df)
risk_prob = booster.predict(dmatrix_input)
print(" Predicted CHD Risk Probability:", round(float(risk_prob[0]), 4))


 Predicted CHD Risk Probability: 0.1499
